In [2]:
from sympy import Symbol, simplify, Integer

In [3]:

class SymbolicValue:
    def __init__(self, initial_value=None, symbolic=False):
        self.symbolic = symbolic
        if symbolic:
            self.value = Symbol('x')
        else:
            self.value = initial_value if initial_value is not None else 0

    def increment(self):
        self.value += 1
        if self.symbolic:
            self.value = simplify(self.value)

    def decrement(self):
        self.value -= 1
        if self.symbolic:
            self.value = simplify(self.value)

    def is_changed(self):
        # fix ze sa pozrie aj ci sa zmenila od posledna
        return self.value != 0 or self.symbolic
    
    def get_optimized_value(self):
        return simplify(self.value) if self.symbolic else self.value

    def reset(self):
        self.value = 0

    def __str__(self):
        return str(self.value)

    
class BrainfuckSymbolicSolver:
    def __init__(self):
        self.tape = [SymbolicValue() for _ in range(30000)]  
        self.pointer = 0
        self.symbolic_state = {}  # to track symbolic values
        self.loop_stack = [] # to keep track of loop
        
    def interpret(self, code):
        # sym interpret
        for char in code:
            self.execute_command(char)

    def execute_command(self, char):
        # Execute a single Brainfuck command
        if char == '>':
            self.pointer += 1
        elif char == '<':
            self.pointer -= 1
        elif char in ['+', '-']:
            self.symbolic_add_sub(char)
        # handel loops

    def symbolic_add_sub(self, char):
        if self.pointer not in self.symbolic_state:
            self.symbolic_state[self.pointer] = SymbolicValue()
        if char == '+':
            self.symbolic_state[self.pointer].increment()
        elif char == '-':
            self.symbolic_state[self.pointer].decrement()


    def optimize(self, code):
        optimized_code = ""
        self.pointer = 0
        self.symbolic_state = {}

        for char in code:
            if char in ['+', '-', '>', '<']:
                self.execute_command(char)
            elif char == '[':
                optimized_code += self.apply_optimizations()
                self.start_loop()
            elif char == ']':
                optimized_code += self.apply_loop_optimizations()
                self.end_loop()
            elif char == '.':
                optimized_code += self.apply_optimizations()
                optimized_code += f"print(chr(tape[{self.pointer}]), end='')\n"
            else:
                optimized_code += self.apply_optimizations()
                optimized_code += char

        # Apply any remaining optimizations at the end
        optimized_code += self.apply_optimizations()
        return optimized_code

    def apply_optimizations(self):
        optimized_code = ""
        for pointer, symbolic_value  in self.symbolic_state.items():
            if symbolic_value.is_changed():
                self.tape[pointer] = symbolic_value
                optimized_code += f"tape[{pointer}] = {symbolic_value.get_optimized_value()}\n"
                # value.reset()

        # self.symbolic_state = {}
        return optimized_code

    def apply_loop_optimizations(self):
        return "[]\n"  # Placeholder for actual loop optimization logic
    
    def optimize_and_convert_to_python(self, bf_filename):
        # with open(bf_filename, 'r') as file:
        #     bf_code = file.read()
        bf_code = bf_filename
        optimized_python_code = self.optimize(bf_code)
        # py_filename = bf_filename.rsplit('.', 1)[0] + '.py'
        py_filename = "test.py"
        with open(py_filename, 'w') as file:
            file.write(self.generate_python_code(optimized_python_code))

        print(f"Python code generated: {py_filename}")
    
    def generate_python_code(self, optimized_python_code):
        python_code_template = """import sys

def main():
    tape = [0] * 30000  # Initialize tape
    pointer = 0
    
    {}
    print()  # New line after program execution

if __name__ == "__main__":
    main()
        """
        # indented_code = '\n'.join('\t' + line for line in optimized_python_code.splitlines())
        indented_code = optimized_python_code.replace('\n', '\n    ')
        return python_code_template.format(indented_code)
 


In [5]:
solver = BrainfuckSymbolicSolver()
solver.interpret("++>+-") 
# solver.optimize()

In [6]:
solver = BrainfuckSymbolicSolver()
solver.optimize_and_convert_to_python("+.++--")

Python code generated: test.py


In [27]:
solver = BrainfuckSymbolicSolver()
optimized_code = solver.optimize("+++++>+++")
print("Optimized code:", optimized_code)

Optimized code: tape[0] = 5
tape[1] = 3



In [28]:
optimized_code = solver.optimize("++>+-")
print("Optimized code:", optimized_code)

Optimized code: tape[0] = 2
tape[1] = 0

